In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest", temperature=0.5, k = 2)

In [ ]:
import bs4
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from transformers import GPT2Tokenizer, GPT2Model
from langchain_chroma import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader = CSVLoader(
    file_path="rag_doc.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["question", "answer"],
    },
)

docs = loader.load()

print(data)

In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=HuggingFaceEmbeddings())

In [ ]:
print(docs[1].page_content)

In [ ]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
retrieved_docs = retriever.invoke("o que é o programa bolsa familia")

page_content = retrieved_docs[0].page_content
answer_start = page_content.find("answer: ") + len("answer: ")
answer = page_content[answer_start:]

print(retrieved_docs[0])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


template = """Você é um modelo de chatbot e irá responder as perguntas baseadas nesse contexto.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
prompt = ChatPromptTemplate.from_template(template)


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Me faça um resumo do programa bolsa familia"))